In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output

upload_output = widgets.Output()
upload_widget = widgets.FileUpload(accept='.ttl', multiple=False)

def on_upload(change):
    upload_output.clear_output()
    with upload_output:
        if upload_widget.value:
            uploaded_file = upload_widget.value[0]  # NEU: Zugriff als Tuple
            file_name = uploaded_file['name']       # NEU: kein ['metadata']['name']
            print(f"✅ You uploaded: {file_name}")
        else:
            print("⚠️ No file uploaded.")

upload_widget.observe(on_upload, names='value')

display(upload_widget, upload_output)


ModuleNotFoundError: No module named 'rdflib'